## Import

In [1]:
import glob
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
# import geoparquet as gpq

from math import *
from shapely.geometry import Point

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
# import dask_geopandas as dg



from pathlib import Path
import os
import glob

import re
from tqdm import tqdm

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Data

In [7]:
%%time
# sa
sa = gpd.read_file('C:/Users/alexd/Dropbox/Work/Repository/FFE/data/shapefile/SA2_input.shp')
sa = sa[['BldgFloor_', 'PopDensity', 'NumberBldg', 'geometry']] # drop column not used in caluclation of Ignition
sa = gpd.GeoDataFrame(sa, crs="EPSG:2193")

# pga 
pga = pd.read_parquet('C:/Users/alexd/Dropbox/Work/Repository/OpenQuake/output/PGAxy.parquet')
pgaGeo = gpd.GeoDataFrame(pga, geometry=gpd.points_from_xy(pga.lon, pga.lat))#, crs="EPSG:2193"))

Wall time: 3min 22s


In [8]:
%%time
del pga
data = gpd.sjoin(pgaGeo, sa, op="within")

Wall time: 25min 13s


In [9]:
# %%time
data.to_fileC:/Users/alexd/Dropboxox/Work/GNS/008_FFE/ProbaFFE/FFE/shapefile/joinSA_PGA.shp')

DriverIOError: Failed to create file /Users/alex/Dropbox/Work/GNS/008_FFE/ProbaFFE/FFE/shapefile\joinSA_PGA.shp: No error

## Ignition calculations

In [ ]:
# for windows
# path_on_windows = PureWindowsPath(filename)

In [4]:
# %%time
# data = gpd.read_file('C:/Users/alexd/Dropbox/Work/GNS/008_FFE/ProbaFFE/FFE/shapefile/joinSA_PGA.shp')

Wall time: 30min 53s


In [35]:
from numba import jit

gmv_PGA = data.gmv_PGA.values
PopDensity = data.PopDensity
BldFloor = data.BldgFloor_.values
NbrBld = data.NumberBldg.values

@jit
def ignition_district(PGAs, PD, SF):
    Pign = np.exp(-6.755 + (8.463 * PGAs) + (98.4e-6 * PD) + (152.3e-6 * SF)) / (1 + np.exp(-6.755 + (8.463 * PGAs) + (98.4e-6 * PD) + (152.3e-6 * SF)))
    return Pign


@jit
def ignition_buildings(IgnDistrict, NBbld):
    PignBld = 1 - np.power(1-IgnDistrict, 1/NBbld)
    return PignBld

In [36]:
%%time
Pignition_district = ignition_district(gmv_PGA, PopDensity, BldFloor)
Pignition_buildings = ignition_buildings(Pignition_district, NbrBld)

Wall time: 3.37 s


In [47]:
data['IgnProb'] = Pignition_buildings
joinSA_PGA = data[['event_id', 'site_id', 'gmv_PGA', 'lon', 'lat', 'geometry','IgnProb']]

In [45]:
data.head(5); data.columns

,event_id,site_id,gmv_PGA,lon,lat,geometry,index_right,BldgFloor_,PopDensity,NumberBldg,IgnProb
__null_dask_index__,,,,,,,,,,,
0,0,0,0.050537,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,2341.79635,2208,1290,0.000002
76,1,0,0.259768,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,2341.79635,2208,1290,0.000014
152,2,0,0.099314,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,2341.79635,2208,1290,0.000004
228,3,0,0.128333,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,2341.79635,2208,1290,0.000005
304,4,0,0.017772,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,2341.79635,2208,1290,0.000002


Index(['event_id', 'site_id', 'gmv_PGA', 'lon', 'lat', 'geometry',
       'index_right', 'BldgFloor_', 'PopDensity', 'NumberBldg', 'IgnProb'],
      dtype='object')

In [46]:
plt.hist(data.site_id.values)

20336384